In [28]:
import numpy as np
from collections import Counter


# Decision tree

In [29]:

class node:
    
    def __init__(self, features = None, threshold = None, left = None, right = None,*, value = None):
        
        self.features = features
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None
        
        
class DecisionTree:
    
    def __init__(self, min_sample_split = 2, max_depth = 100, n_features = None ):
        
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
        
        
    def fit(self, X, Y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X,Y)
        
        
    def _grow_tree(self, X, Y, depth = 0):
        
        no_of_samples, no_of_features  = X.shape
        no_of_labels = len(np.unique(Y))
        
        # check stopping criteria
        if (depth >= self.max_depth or no_of_labels == 1 or no_of_samples < self.min_sample_split):
            
            leaf_value = self.most_common_label(Y)
            return node(value = leaf_value )
        
        feature_idx = np.random.choice(no_of_features, self.n_features, replace =False)
        
        # find the best split
        best_feature, best_threshold = self.best_split(X, Y, feature_idx)
        
        
        # create a child node
        left_idx, right_idx = self.split(X[:, best_feature], best_threshold)
        left = self._grow_tree(X[left_idx, :], Y[left_idx], depth+1)
        right = self._grow_tree(X[right_idx, :], Y[right_idx], depth+1)
        return node(best_feature, best_threshold, left, right)
        
        
    def best_split(self, X, Y, feature_idx):
        
        best_gain = -1
        split_index, split_threshold = None, None
        
        for feat in feature_idx:
            
            X_column = X[: , feat]
            thresholds = np.unique(X_column)
            
            for thr in thresholds:
                gain = self.information_gain(Y, X_column, thr)
                
                if(gain > best_gain):
                    best_gain = gain
                    split_index = feat
                    split_threshold = thr
                    
        return split_index, split_threshold
    
    def information_gain(self, Y, X_column, threshold):
        
        # parent entropy
        parent_entropy = self.entropy(Y)
        
        
        
        # create children
        left_idx, right_idx = self.split(X_column, threshold)
        
        if (len(left_idx) == 0) or (len(right_idx) == 0):
            return 0
        
        # calculate the weighted entropy of children
        n = len(Y)
        n_l, n_r = len(left_idx), len(right_idx)
        e_l, e_r = self.entropy(Y[left_idx]), self.entropy(Y[right_idx])
        child_entropy = (n_l/n)*e_l + (n_r/n)*e_r
        
        
        # calculate information gain
        info_gain = parent_entropy - child_entropy
        return info_gain
    
    
      
    def split(self, X_column, split_thresh):
        
        left_idx = np.argwhere(X_column <= split_thresh).flatten()
        right_idx = np.argwhere(X_column > split_thresh).flatten()
        return left_idx, right_idx
        
    
    def entropy(self, Y):
        
        hist = np.bincount(Y)
        ps = hist/len(Y)
        return -np.sum([p*np.log(p) for p in ps if p>0])
        
        
    
    def most_common_label(self, Y):
        counter = Counter(Y)
        value = counter.most_common(1)[0][0]
        return value
    
    def predict(self, X):
        return np.array([self.traverse_tree(x, self.root) for x in X])
    
    def traverse_tree(self, x, node):
        
        if node.is_leaf_node():
            return node.value
          
        
        if x[node.features]<=node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)

# Random Forest

In [49]:
class RandomForest:
    
    def __init__(self, n_trees = 10, max_depth = 10, min_sample_split = 2, n_feature = None):
        
        # initializing variables
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.n_feature = n_feature
        self.trees= []
        
    def fit(self, X, Y):
        self.trees = [] # creating an empty list to store all of the trees
        for i in range(self.n_trees):
            
            # Calling decision trees class to create tree
            tree = DecisionTree(min_sample_split=self.min_sample_split, max_depth = self.max_depth, n_features = self.n_feature)
            
            # Function for random samples with replacement
            X_sample, Y_sample = self.bootstrap_sample(X,Y) 
            
            # Creating a tree for one particular sample
            tree.fit(X_sample,Y_sample) 
            
            # adding the tree in the list
            self.trees.append(tree)  
            
    def bootstrap_sample(self, X,Y):
        
        # Choosing random index numbers from the data
        n_shape = X.shape[0]
        sample_idx = np.random.choice(n_shape, n_shape, replace = True)
        
        # return the sample data through the chosen indexes
        return X[sample_idx], Y[sample_idx]
    
    
    
    def most_common_label(self, y):  
        counter = Counter(y)
        most_common_val = counter.most_common(1)[0][0]
        return most_common_val
    
    def predict(self, X):
        
        #predictiong y values for each X rows from each trees and then storing them in arrays
        predict = np.array([tree.predict(X) for tree in self.trees])
        
        #Result of previous computation [[tree1 prediction], [tree2 prediction]...] = [[X1,X2,X3,X4], [X1,X2,X3,X4]...] 
        
        #Now changing the axes of our array to get all the predictions for one X the first list
        predict1 = np.swapaxes(predict, 0,1)
        
         #Result: [[X1,X1],[X2,X2],[X3,X3],[X4,X4]]
        
        # predicting most common y value for each array
        label_prediction = np.array([self.most_common_label(p) for p in predict1])
        
        return label_prediction
        
            

In [50]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [52]:
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
Y = breast_cancer.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)

def accuracy(y_pred, y_test):
    
    accuracy = np.sum(y_pred == y_test)/len(y_test)
    return accuracy

model = RandomForest(n_trees = 20)
model.fit(X_train, Y_train)

predictions_data = model.predict(X_test)
accuracy(predictions_data, Y_test)

0.9210526315789473